In [1]:
%matplotlib notebook
import os
from adm_library import *

## 0 Parameter definitions

### 0.1 Ingesting the [X,Y,Z,Rx,Ry] values from Tyler's spreadsheet

In [2]:
#fname = 'files/Cal5DoF05222023-post FDPR V3.xlsx'
sheetname = 'Input for Tyler analysis'
fname = 'files/5DoF inputs - BA5C2.xlsx'

output_filename = 'Poses - BA5C4 FDPR update from BA5C2 - Evan.xlsx'

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:N')
pose_actual = spreadsheet.iloc[0:5,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]
pose_encoders = spreadsheet.iloc[0:5,7:13]
pose_encoders = pose_encoders.set_index('Name.1').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
                X          Y          Z        Rx         Ry
Name                                                       
PR1     2.163165   0.731032   0.966148  0.025574   0.002490
PR2  -108.342474 -46.323188  23.595026 -4.904773  10.653399
PR3   112.472837 -46.012767  23.450489 -4.875768 -10.627238
PR4   104.321314  59.133634  32.073318  5.290663 -10.350558
PR5  -100.186324  58.940085   32.03125  5.275375  10.371512 

Provided 5DOF encoder values: 
                  X          Y          Z        Rx         Ry
Name.1                                                       
PR1       2.138863   0.770344   0.950754  0.026304   0.004572
PR2    -108.249306 -46.371321  23.478986 -4.904729  10.652422
PR3     112.423147 -45.708420  23.548210 -4.874262 -10.626989
PR4     104.127459  59.355654  32.271674  5.295555 -10.348123
PR5    -100.245410  58.811642  32.026732  5.274800  10.372370


### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [3]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [4]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

### 0.3 Form a dataframe with the values supplied so far

In [5]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA',['Rx','Ry','Rz']] = sMPA_angle_to_WCS_deg.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

,X,Y,Z,Rx,Ry,color,Rz,uvec_X,uvec_Y,uvec_Z
Name,,,,,,,,,,
PR1,2.163165,0.731032,0.966148,0.025574,0.002490,crimson,NaN,0.056155,248.273543,1268.046565
PR2,-108.342474,-46.323188,23.595026,-4.904773,10.653399,crimson,NaN,238.871363,350.194693,1220.608863
PR3,112.472837,-46.012767,23.450489,-4.875768,-10.627238,crimson,NaN,-238.291518,349.606727,1220.890714
PR4,104.321314,59.133634,32.073318,5.290663,-10.350558,crimson,NaN,-232.156166,128.735590,1264.560204
PR5,-100.186324,58.940085,32.03125,5.275375,10.371512,crimson,NaN,232.621007,129.064383,1264.441260
sMask,1.995,115.232021,588.279458,NaN,NaN,g,NaN,NaN,NaN,NaN
sMPA,2.04,248.465,1268.826,-24.297100,0.007400,purple,0.0746,0.000129,0.411468,0.911424


# 1 Updating poses based on results from FDPR

### 1.1 Reading in offset data from the FDPR team's spreadsheet.

In [6]:
#These data will be used to update the position of the calculated endpoints in the sMPA frame
shift_data = pd.read_excel('files/FDPR shifts - sandbox.xlsx',sheet_name='PSF Shifts',skiprows=0,usecols='C:K')
shift_data = shift_data.iloc[0:6]
shift_data.set_index('camera',inplace=True)
print(shift_data)

        det normal dz (um)  chief ray dz (um)  row  ...  dy (px)  dx (um)  dy (um)
camera                                              ...                           
PR1                    NaN                 20  NaN  ...      NaN       37        0
PR3                    NaN                -11  NaN  ...      NaN       -7       78
PR4                    NaN                 38  NaN  ...      NaN        9       60
PR5                    NaN                -15  NaN  ...      NaN      -59       -5
PR2                    NaN                 28  NaN  ...      NaN      -32       80

[5 rows x 8 columns]


#### 1.1.1 Calculate some transformation variables necessary for converting between the 5DOF and sMPA frame

In [7]:
translation_to_sMPA = df.loc['sMPA',['X','Y','Z']]
rotation_from_sMPA_to_5DOF = R.from_matrix(rotmat_from_2vec(np.array([0,0,1]),df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']].values.astype(float)))

## Scratch

### 1.2 Calculate the new best-fit poses that minimize the deltas observed by the FDPR team (with 5DOF optimization)

In [8]:
df_after_fitting_FDPR_shifts,endpoints_residuals_sMPA,best_fit_deltas = pose_update_with_FDPR_results(df,shift_data,focal_length,GSA_angle_WCS_deg,translation_to_sMPA,rotation_from_sMPA_to_5DOF)
print(np.std(endpoints_residuals_sMPA))

Residuals (in sMPA frame) baseline: 
       endpt_X   endpt_Y   endpt_Z
PR1 -0.037001 -0.003885 -0.019603
PR2  0.026928 -0.080466  0.006446
PR3  0.005073 -0.068107  0.042483
PR4 -0.002094 -0.058482 -0.012495
PR5  0.061694  0.006122  0.012583 

Best-fit (X,Y,Z,Rx,Ry) deltas to apply to real-space poses: 
 [ 0.0067  0.0106 -0.0041  0.0024  0.0002] 

Residuals (in sMPA frame) after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
      endpt_X  endpt_Y  endpt_Z
PR1   -0.048    0.038   -0.026
PR2    0.016   -0.040   -0.003
PR3   -0.006   -0.028    0.031
PR4   -0.013   -0.017   -0.015
PR5    0.051    0.048    0.012 

endpt_X    0.032684
endpt_Y    0.035935
endpt_Z    0.020073
dtype: float64


### 1.2 (alternate) Calculate the new best-fit poses that minimize the deltas observed by the FDPR team (with 6DOF optimization)

In [9]:
df_after_fitting_FDPR_shifts,endpoints_residuals_sMPA,best_fit_deltas = pose_update_with_FDPR_results_v2(df,shift_data,focal_length,GSA_angle_WCS_deg,translation_to_sMPA,rotation_from_sMPA_to_5DOF)
print(np.std(endpoints_residuals_sMPA))

Residuals baseline: 
       endpt_X   endpt_Y   endpt_Z
PR1 -0.037001 -0.003885 -0.019603
PR2  0.026928 -0.080466  0.006446
PR3  0.005073 -0.068107  0.042483
PR4 -0.002094 -0.058482 -0.012495
PR5  0.061694  0.006122  0.012583 

Best-fit 6DoF deltas to apply to real-space poses: 
 [ 0.0066  0.0105 -0.0041  0.0024  0.0008  0.0034] 

Residuals after incorporating the best-fit 6DoF shift to all poses: 
      endpt_X  endpt_Y  endpt_Z
PR1   -0.048    0.038   -0.026
PR2    0.023   -0.054   -0.001
PR3    0.001   -0.015    0.029
PR4   -0.020   -0.003   -0.017
PR5    0.044    0.034    0.015 

endpt_X    0.032019
endpt_Y    0.033917
endpt_Z    0.020012
dtype: float64


### 1.3 Convert new best-fit poses to encoder space

In [10]:
df_after_fitting_FDPR_shifts_in_encoder_space = pd.DataFrame(columns=['X','Y','Z','Rx','Ry'],index=df_after_fitting_FDPR_shifts.index)
for pose in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
    df_after_fitting_FDPR_shifts_in_encoder_space.loc[pose] = calculate_encoders_from_5DOF(pd.DataFrame(df_after_fitting_FDPR_shifts.loc[pose]).T).squeeze()

print('New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: \n',df_after_fitting_FDPR_shifts_in_encoder_space,'\n')
print('Baseline pose encoders (displayed again here just for reference): \n', pose_encoders)

New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
               X          Y          Z        Rx         Ry
PR1    2.145434   0.780755   0.946776  0.028667   0.004757
PR2  -108.24273  -46.36087  23.474953 -4.902981  10.652322
PR3  112.429691 -45.697969  23.544233 -4.871279 -10.627087
PR4  104.134009  59.366032  32.267756   5.29854 -10.347627
PR5 -100.238831  58.822019  32.022762  5.276543  10.372864 

Baseline pose encoders (displayed again here just for reference): 
                  X          Y          Z        Rx         Ry
Name.1                                                       
PR1       2.138863   0.770344   0.950754  0.026304   0.004572
PR2    -108.249306 -46.371321  23.478986 -4.904729  10.652422
PR3     112.423147 -45.708420  23.548210 -4.874262 -10.626989
PR4     104.127459  59.355654  32.271674  5.295555 -10.348123
PR5    -100.245410  58.811642  32.026732  5.274800  10.372370


# 2 Write the new poses to an Excel file in the current directory

In [11]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel(output_filename,'',update_type='FDPR',baseline_filepath=fname,
                             columns=columns,GSA_angle_WCS_deg=GSA_angle_WCS_deg,rigid_body_correction=best_fit_deltas,
                             df=df,df_encoders=pose_encoders,df_update=df_after_fitting_FDPR_shifts,df_update_encoders=df_after_fitting_FDPR_shifts_in_encoder_space,
                            focal_length=focal_length)

**Writing to Excel complete.**
**Filename:  Poses - BA5C4 FDPR update from BA5C2 - Evan.xlsx
